In [30]:
import torch
from torch import nn
from d2l import torch as d2l
from train_epoch.train import train_ch6

In [40]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

In [41]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [42]:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        # 全连接层部分
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

In [43]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


In [44]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [45]:
lr, num_epochs, batch_size = 0.05, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu(),save_path="models/vgg")

new models
training on cuda:0


Epoch 1/10: 100%|██████████| 60000/60000 [00:23<00:00, 2538.19img/s, loss=1.0766, train_acc=0.6094]


epoch 0, loss 1.077, train acc 0.609, test acc 0.826


Epoch 2/10: 100%|██████████| 60000/60000 [00:23<00:00, 2571.48img/s, loss=0.4005, train_acc=0.8519]


epoch 1, loss 0.400, train acc 0.852, test acc 0.868


...

epoch 7, loss 0.206, train acc 0.924, test acc 0.916


Epoch 9/10: 100%|██████████| 60000/60000 [00:23<00:00, 2579.89img/s, loss=0.1914, train_acc=0.9287]


epoch 8, loss 0.191, train acc 0.929, test acc 0.910


Epoch 10/10: 100%|██████████| 60000/60000 [00:23<00:00, 2562.06img/s, loss=0.1781, train_acc=0.9344]


epoch 9, loss 0.178, train acc 0.934, test acc 0.922
Saved best models checkpoint to models/vgg\0507232614\best.ckpt with test accuracy 0.922
loss 0.178, train acc 0.934, test acc 0.922
2970.6 examples/sec on cuda:0
